# Or... A speed test

In [8]:
block_size = 50000
hdf5_fname = '../raw/total-3L.h5'
vcf_fname = '../raw/total-3L.vcf.gz'

In [9]:
import cProfile

import dask.array as da
import numpy as np
import tables

import vcf

# Sequential HDF5 vs Sequential VCF

In [10]:
#Preamble to get the maximum position
store = tables.open_file(hdf5_fname, 'r')
pos_array = store.get_node('/3L/variants/POS').iterrows()

num_snps = pos_array.nrows
max_pos = pos_array.read(num_snps - 1)[0]  # [-1] notation not yet supported
store.close()

In [11]:
#1-index vs 0-index
def compute_bin_index(position):
    return (position - 1) // block_size

## HDF5

In [6]:
def compute_snps_per_bin_hdf5(in_memory=False):
    # lets save memory, not a lot in this case, but an example
    count_snps = np.zeros(1 + max_pos // block_size, dtype=np.uint16 if block_size < 65536 else np.uint32)
    store = tables.open_file(hdf5_fname, 'r')
    if in_memory:
        pos_iter = store.get_node('/3L/variants/POS').read()
    else:
        pos_iter = store.get_node('/3L/variants/POS').iterrows()
    for pos in pos_iter:
        count_snps[compute_bin_index(pos)] += 1
    store.close()
    return count_snps

In [37]:
#We can do in-memory load
%time compute_snps_per_bin_hdf5(True)

CPU times: user 35.8 s, sys: 1.86 s, total: 37.6 s
Wall time: 37.7 s


array([   44,   922,   947,  1508,    16,   367,  1235,  1570,   193,
         255,   471,     0,   608,   844,  1056,  1619,  1869,  2161,
         513,   790,   746,   122,  1363,   560,  1118,  2746,   964,
         606,     0,   796,   556,  2965,  3118,  1341,  2240,  2875,
        2028,  3171,  2948,  4471,  5361,  4225,  4150,  4391,  4521,
        4125,  1693,  3975,  3986,  3742,  2866,  3822,  4708,  3372,
        2529,  3106,  4640,  4710,  6672,  2866,  3729,  4116,  3987,
        4380,  4710,  3915,  4001,  4106,  5719,  5161,  5011,  5496,
        4526,  4518,  4488,  3837,  4584,  4038,  5364,  2917,  3237,
        3837,  3691,  2930,  3083,   917,   368,   156,   705,    83,
           2,   112,   126,     0,    37,   223,  3992,  3508,  2452,
         405,  3070,  3722,  4343,  5154,  4086,  3949,  5600,  5617,
        5373,  5003,  4531,  3858,  5463,  5895,  5544,  4980,  5603,
        3133,  4859,  6090,  4047,  5047,  3224,  3732,  4010,  6464,
        3667,  4161,

In [38]:
%time compute_snps_per_bin_hdf5(False)

CPU times: user 49.3 s, sys: 24 ms, total: 49.3 s
Wall time: 49.3 s


array([   44,   922,   947,  1508,    16,   367,  1235,  1570,   193,
         255,   471,     0,   608,   844,  1056,  1619,  1869,  2161,
         513,   790,   746,   122,  1363,   560,  1118,  2746,   964,
         606,     0,   796,   556,  2965,  3118,  1341,  2240,  2875,
        2028,  3171,  2948,  4471,  5361,  4225,  4150,  4391,  4521,
        4125,  1693,  3975,  3986,  3742,  2866,  3822,  4708,  3372,
        2529,  3106,  4640,  4710,  6672,  2866,  3729,  4116,  3987,
        4380,  4710,  3915,  4001,  4106,  5719,  5161,  5011,  5496,
        4526,  4518,  4488,  3837,  4584,  4038,  5364,  2917,  3237,
        3837,  3691,  2930,  3083,   917,   368,   156,   705,    83,
           2,   112,   126,     0,    37,   223,  3992,  3508,  2452,
         405,  3070,  3722,  4343,  5154,  4086,  3949,  5600,  5617,
        5373,  5003,  4531,  3858,  5463,  5895,  5544,  4980,  5603,
        3133,  4859,  6090,  4047,  5047,  3224,  3732,  4010,  6464,
        3667,  4161,

In [5]:
def compute_snps_per_bin_vcf():
    # lets save memory, not a lot in this case, but an example
    count_snps = np.zeros(1 + max_pos // block_size, dtype=np.uint16 if block_size < 65536 else np.uint32)
    vcf_file = vcf.Reader(filename=vcf_fname)
    for rec in vcf_file:
        count_snps[compute_bin_index(rec.POS)] += 1
    store.close()
    return count_snps

In [ ]:
%time compute_snps_per_bin_vcf()

In [13]:
#TODO: Do partial function application of the above

# Dask with VCF indexes

In [ ]:
store = tables.open_file(hdf5_fname, 'r')
pos_iter = store.get_node('/3L/variants/POS')
da_pos = da.from_array(pos_iter)
store.close()


# Sparse matrix